In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('All done')

All done


### Dataframe1

read file

In [2]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.columns=['PostalCode','Borough','Neighborhood']
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Drop Borough == 'Not assigned' cells

In [3]:
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


Merge cells with same Postcode and Borough

## Answer to Q1

In [4]:
df1= df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x))
df1=df1.reset_index()
df1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


For those neighborhood are not assigned, assign them the same value as Borough

In [5]:
index_=df1[df1['Neighborhood']=='Not assigned'].index
df1.loc[index_,'Neighborhood']=df1.loc[index_,'Borough']


In [6]:
df1.shape

(103, 3)

### Dataframe2

In [7]:
geo=pd.read_csv('https://cocl.us/Geospatial_data')
geo.columns=['PostalCode','Latitude','Longitude']
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Answer to Q2

In [8]:
new=pd.merge(df1,geo,on='PostalCode')
new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


### Explore and cluster neighorhoods in Toronto

Only check for the boroughs that have the word 'Toronto':

In [11]:
CLIENT_ID = 'T3PI1WRTAKDAQWNGUGX534H5C0A2SHAPQNOSRLSN5MQQ2TSD' # your Foursquare ID
CLIENT_SECRET = 'Z2CI3T12IZBUDY5AYRDEISVMWHEZ2W3LVE4VCGLUFWKZWSRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T3PI1WRTAKDAQWNGUGX534H5C0A2SHAPQNOSRLSN5MQQ2TSD
CLIENT_SECRET:Z2CI3T12IZBUDY5AYRDEISVMWHEZ2W3LVE4VCGLUFWKZWSRG


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Only consider venues in downtown Toronto

Explore the venues within a radius of 500 meters based on postal code

In [28]:
downtown=new[new['Borough'].str.find('Toronto')>0]
downtown


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [29]:
downtown_venues=getNearbyVenues(names=downtown['Neighborhood'],
                                latitudes=downtown['Latitude'],
                                longitudes=downtown['Longitude'])
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [30]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 237 uniques categories.


In [34]:
downtown_onehot=pd.get_dummies(downtown_venues[['Venue Category']],prefix='',prefix_sep='')
downtown_onehot['Neighborhood']=downtown_venues['Neighborhood']
# fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
# downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.set_index('Neighborhood',inplace=True)
downtown_onehot.reset_index(drop=False,inplace=True)
downtown_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by postalcode and taking the mean of the frequency of occurence of each category

In [35]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.020000,0.000000,...,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.0625,0.1875,0.0625,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.011628,0.000000,...,0.00000,0.00,0.011628,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.011628
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.055556,0.000000,0.055556,0.011111,0.000000,0.000000,0.011111,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011628,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.011628,0.000000,...,0.00000,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.011628,0.000000,0.023256


Print each postal code along with the top 5 most common venues

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

downtown_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Sushi Restaurant,Burger Joint,Asian Restaurant,Restaurant,Thai Restaurant,Pizza Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Farmers Market,Seafood Restaurant,Cheese Shop,Café,Park
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Pet Store,Bar,Italian Restaurant,Bakery,Stadium,Climbing Gym,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Smoke Shop,Spa,Restaurant,Burrito Place,Skate Park,Auto Workshop,Fast Food Restaurant,Farmers Market,Butcher
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Boutique,Harbor / Marina,Bar,Coffee Shop,Plane,Boat or Ferry,Sculpture Garden,Rental Car Location,Airport Terminal


Cluster Neighborhood

In [44]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [45]:
# add clustering labels
downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Dessert Shop,Diner,Pub,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Liquor Store,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Intersection,Brewery,Food & Drink Shop,Pub
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Diner,Pet Store,Park
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


## Answer to Q3

In [46]:
address='Toronto,Canada'
geolocator=Nominatim(user_agent='to_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


In [48]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [49]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,0,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
41,M4K,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Dessert Shop,Diner,Pub,Caribbean Restaurant
42,M4L,0,Park,Sandwich Place,Liquor Store,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Intersection,Brewery,Food & Drink Shop,Pub
43,M4M,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Diner,Pet Store,Park
45,M4P,0,Gym,Asian Restaurant,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Hotel,Park,General Entertainment,Dessert Shop
46,M4R,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Bagel Shop,Burger Joint,Café,Chinese Restaurant,Dessert Shop,Diner
47,M4S,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Gym,Coffee Shop,Café,Sushi Restaurant,Chinese Restaurant,Bowling Alley
49,M4V,0,Light Rail Station,Coffee Shop,Pub,American Restaurant,Bagel Shop,Restaurant,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Supermarket
51,M4X,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Outdoor Sculpture
52,M4Y,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


In [23]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,1,Park,Playground,Trail,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [24]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,M4X,2,Coffee Shop,Italian Restaurant,Park,Pizza Place,Bakery,Market,Restaurant,Café,Pub,Outdoor Sculpture
52,M4Y,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,M5A,2,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Performing Arts Venue,Chocolate Shop
54,M5B,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Burger Joint,Diner,Plaza,Pizza Place
55,M5C,2,Coffee Shop,Café,Restaurant,Hotel,Bakery,Clothing Store,Beer Bar,Cosmetics Shop,Cocktail Bar,Breakfast Spot
56,M5E,2,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Bakery,Steakhouse,Cheese Shop,Café,Japanese Restaurant
57,M5G,2,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Chinese Restaurant,Middle Eastern Restaurant,Juice Bar,Bar
58,M5H,2,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Restaurant,Burger Joint,Gym
59,M5J,2,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,Music Venue
60,M5K,2,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Deli / Bodega,Bar,American Restaurant,Gastropub,Italian Restaurant
